<a href="https://colab.research.google.com/github/jocelynbaduria/cmpe-257_Machine_Learning/blob/main/Politifact_4_11_4_21PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup 
import pandas as pd
import requests
import urllib.request
import time

In [ ]:
# Create a list to stored the dataset
authors = []
dates = []
statements = []
sources = []
targets = []

In [ ]:
# Webcrawler
def crawl_politifact(page_number):
  page = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+page
  # Request to the website
  web = requests.get(URL)
  # Parse the text from the website
  soup = BeautifulSoup(web.text, 'html.parser')
  # Get the location of information in the web for the author and the date ( tag and classes )
  statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})
  statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) 
  statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})
  target = soup.find_all('div', attrs={'class':'m-statement__meter'}) 

  #loop through the footer class m-statement__footer to get the date and author
  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month+' '+day+' '+year
    dates.append(date)
    authors.append(full_name)

  #Loop through the div m-statement__quote to get the link
  for i in statement_quote:
    link2 = i.find_all('a')
    statements.append(link2[0].text.strip())

 #Loop through the div m-statement__meta to get the source
  for i in statement_meta:
    link3 = i.find_all('a') #Source
    source_text = link3[0].text.strip()
    sources.append(source_text)

  #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
  for i in target:
    fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
    targets.append(fact)

In [ ]:
#Loop through 'n-1' webpages to scrape the data
n=2
for i in range(1, n):
  crawl_politifact(i)

In [ ]:
#Create a new dataFrame 
data = pd.DataFrame(columns = ['author',  'statement', 'source', 'date', 'target']) 
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets
#Show the data set
data

,author,statement,source,date,target
0,Jill Terreri,"""In fact, the data coming in from some states ...",Liz Krueger,"• April 9,",half-true
1,Michael Majchrowicz,“56 million defenseless people ‘rounded up and...,Instagram posts,"April 9, 2021",false
2,Jon Greenberg,Says Officer Derek Chauvin’s knee was “not on ...,Instagram posts,"April 9, 2021",false
3,Tom Kertscher,"“Kids at the border: 2,600 highest number Trum...",Brandon Tatum,"April 9, 2021",half-true
4,Madison Czopek,"“Covid-does NOT exist AS A VIRUS,” and Russian...",Instagram posts,"April 9, 2021",pants-fire
5,Brandon Mulder,“Nothing has changed in the election code (und...,Dan Patrick,"April 9, 2021",pants-fire
6,Louis Jacobson,"“You go to a gun show, you can buy whatever yo...",Joe Biden,"April 8, 2021",barely-true
7,Tom Kertscher,"""New poll indicates” Joe Biden’s “approval at ...",Facebook posts,"April 8, 2021",pants-fire
8,Madeline Heim,Says it doesn't make sense for young adults to...,Tom Tiffany,"April 8, 2021",barely-true
9,Tom Kertscher,“Former Las Vegas Raiders offensive tackle Bru...,Facebook posts,"April 7, 2021",false


In [ ]:
#Create a function to get a binary number from the target
def getBinaryNumTarget(text):
  if text == 'true':
    return 1
  else:
    return 0

In [ ]:
#Create a function to get only true or false values from the target
def getBinaryTarget(text):
  if text == 'true':
    return 'REAL'
  else:
    return 'FAKE'

In [ ]:
#Store the data in the dataframe
data['BinaryTarget'] = data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)

In [ ]:
#Show the data
data

,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Jill Terreri,"""In fact, the data coming in from some states ...",Liz Krueger,"• April 9,",half-true,FAKE,0
1,Michael Majchrowicz,“56 million defenseless people ‘rounded up and...,Instagram posts,"April 9, 2021",false,FAKE,0
2,Jon Greenberg,Says Officer Derek Chauvin’s knee was “not on ...,Instagram posts,"April 9, 2021",false,FAKE,0
3,Tom Kertscher,"“Kids at the border: 2,600 highest number Trum...",Brandon Tatum,"April 9, 2021",half-true,FAKE,0
4,Madison Czopek,"“Covid-does NOT exist AS A VIRUS,” and Russian...",Instagram posts,"April 9, 2021",pants-fire,FAKE,0
5,Brandon Mulder,“Nothing has changed in the election code (und...,Dan Patrick,"April 9, 2021",pants-fire,FAKE,0
6,Louis Jacobson,"“You go to a gun show, you can buy whatever yo...",Joe Biden,"April 8, 2021",barely-true,FAKE,0
7,Tom Kertscher,"""New poll indicates” Joe Biden’s “approval at ...",Facebook posts,"April 8, 2021",pants-fire,FAKE,0
8,Madeline Heim,Says it doesn't make sense for young adults to...,Tom Tiffany,"April 8, 2021",barely-true,FAKE,0
9,Tom Kertscher,“Former Las Vegas Raiders offensive tackle Bru...,Facebook posts,"April 7, 2021",false,FAKE,0


In [50]:
#Store the data to a CSV file
data.to_csv('political_fact_checker.csv')